In [1]:
import pandas as pd
import torch
import numpy as np
import ast
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from transformers import AdamW
from sklearn.model_selection import train_test_split
from pandarallel import pandarallel

In [2]:
dataRew=pd.read_csv("../Dataset/datiClean.csv")

In [3]:
dataRew.drop(['review_date','movie_id','user_id','rating','review_summary'],axis=1,inplace=True)

In [4]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [ ]:
dataRew

,is_spoiler,review_text,clean_review
0,True,"In its Oscar year, Shawshank Redemption (writt...","['oscar', 'year', 'shawshank', 'redemption', '..."
1,True,The Shawshank Redemption is without a doubt on...,"['shawshank', 'redemption', 'without', 'doubt'..."
2,True,I believe that this film is the best story eve...,"['believe', 'film', 'best', 'story', 'ever', '..."
3,True,"**Yes, there are SPOILERS here**This film has ...","['yes', 'spoiler', 'film', 'emotional', 'impac..."
4,True,At the heart of this extraordinary movie is a ...,"['heart', 'extraordinary', 'movie', 'brilliant..."
...,...,...,...
573908,False,"Go is wise, fast and pure entertainment. Assem...","['go', 'wise', 'fast', 'pure', 'entertainment'..."
573909,False,"Well, what shall I say. this one´s fun at any ...","['well', 'shall', 'say', 'one', 'fun', 'rate',..."
573910,False,"Go is the best movie I have ever seen, and I'v...","['go', 'best', 'movie', 'ever', 'seen', 'seen'..."
573911,False,Call this 1999 teenage version of Pulp Fiction...,"['call', '1999', 'teenage', 'version', 'pulp',..."


In [5]:
dataRew["clean_review"]=dataRew.loc[:,"clean_review"].parallel_apply(ast.literal_eval)


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [17]:
x=dataRew['clean_review']
y=dataRew['is_spoiler']

In [18]:
## Stratify bilancia i dataset 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,random_state=42)

In [19]:
X_train

94625     [maltese, falcon, film, noir, based, novel, ti...
89585     [water, scarce, resource, dispensed, made, ava...
187840    [really, expecting, lot, given, hype, wow, ove...
265694    [movie, awful, scary, character, dull, plot, t...
125407    [town, excellent, film, flaw, however, one, be...
                                ...                        
174403    [hack, job, review, film, released, admit, exp...
274593    [film, like, american, psycho, one, attracts, ...
514599    [braveheart, currently, listed, imdb, list, to...
203500    [predictable, uninteresting, transparently, ge...
221631    [okay, first, let, say, movie, may, seem, like...
Name: clean_review, Length: 459130, dtype: object

### Bert


In [21]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
def cleanFullrew(text):
    text=" ".join(text)
    return text


In [23]:
prova=dataRew.loc[1,"clean_review"]

In [24]:
dataRew["clean_review_text"]=dataRew["clean_review"].parallel_apply(cleanFullrew)

In [25]:
dataRew

,is_spoiler,review_text,clean_review,clean_review_text
0,True,"In its Oscar year, Shawshank Redemption (writt...","[oscar, year, shawshank, redemption, written, ...",oscar year shawshank redemption written direct...
1,True,The Shawshank Redemption is without a doubt on...,"[shawshank, redemption, without, doubt, one, b...",shawshank redemption without doubt one brillia...
2,True,I believe that this film is the best story eve...,"[believe, film, best, story, ever, told, film,...",believe film best story ever told film tell ti...
3,True,"**Yes, there are SPOILERS here**This film has ...","[yes, spoiler, film, emotional, impact, find, ...",yes spoiler film emotional impact find hard wr...
4,True,At the heart of this extraordinary movie is a ...,"[heart, extraordinary, movie, brilliant, indel...",heart extraordinary movie brilliant indelible ...
...,...,...,...,...
573908,False,"Go is wise, fast and pure entertainment. Assem...","[go, wise, fast, pure, entertainment, assembli...",go wise fast pure entertainment assembling exc...
573909,False,"Well, what shall I say. this one´s fun at any ...","[well, shall, say, one, fun, rate, three, plot...",well shall say one fun rate three plotlines or...
573910,False,"Go is the best movie I have ever seen, and I'v...","[go, best, movie, ever, seen, seen, lot, movie...",go best movie ever seen seen lot movie read re...
573911,False,Call this 1999 teenage version of Pulp Fiction...,"[call, 1999, teenage, version, pulp, fiction, ...",call 1999 teenage version pulp fiction whateve...


In [28]:
def tokenize_text(text):
    from transformers import BertTokenizer
    tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
    return tokenizer(text, padding=True, truncation=True, return_tensors="pt")



In [29]:
# Applica la funzione in parallelo ai dati
tokenized_data = dataRew['clean_review_text'].parallel_apply(tokenize_text)

In [ ]:
tokenized_data